In [1]:
import pandas as pd
import numpy as np
import requests
import lxml.html
import pandas as pd
import numpy as np
from time import sleep
import xml.etree.ElementTree as ET
import itertools
from itertools import compress

In [2]:
all_info = pd.read_csv('all_info.csv')
all_info.drop('Unnamed: 0',axis = 1, inplace = True)
all_info.head()

,reviewer,review_title,text,review_rating,ID,link,rating,total_reviews,title
0,rw4785991,It's the movie Big With superpowers!,"Character development is horrible, fast paced ...",1,tt0448115,https://www.imdb.com/title/tt0448115/?ref_=adv...,7.6,1095.0,Shazam!
1,rw4784318,The last third of this movie was directed by s...,There is an excuse for sitting through the fir...,1,tt0448115,https://www.imdb.com/title/tt0448115/?ref_=adv...,7.6,1095.0,Shazam!
2,rw4768014,"Poor storyline, too childish and boring",Can't believe anyone would give this movie 10/...,1,tt0448115,https://www.imdb.com/title/tt0448115/?ref_=adv...,7.6,1095.0,Shazam!
3,rw4782869,The worst superhero movie of all time,The idea of the film is essentially copied/pla...,1,tt0448115,https://www.imdb.com/title/tt0448115/?ref_=adv...,7.6,1095.0,Shazam!
4,rw4778480,DC's power ranger,This is my first review in IMDB as I wanted to...,1,tt0448115,https://www.imdb.com/title/tt0448115/?ref_=adv...,7.6,1095.0,Shazam!


In [3]:
all_info.link.values[1]

'https://www.imdb.com/title/tt0448115/?ref_=adv_li_tt'

In [4]:
users = pd.read_csv('user_links.csv')
users.head()

,index,user rating list,user review list
0,0,https://www.imdb.com/user/ur62723141/ratings,https://www.imdb.com/user/ur62723141/reviews
1,1,https://www.imdb.com/user/ur31593185/ratings,https://www.imdb.com/user/ur31593185/reviews
2,2,https://www.imdb.com/user/ur84265114/ratings,https://www.imdb.com/user/ur84265114/reviews
3,3,https://www.imdb.com/user/ur60501911/ratings,https://www.imdb.com/user/ur60501911/reviews
4,4,https://www.imdb.com/user/ur101627855/ratings,https://www.imdb.com/user/ur101627855/reviews


In [5]:
users.tail()

,index,user rating list,user review list
139,994,https://www.imdb.com/user/ur91555233/ratings,https://www.imdb.com/user/ur91555233/reviews
140,995,https://www.imdb.com/user/ur68981404/ratings,https://www.imdb.com/user/ur68981404/reviews
141,996,https://www.imdb.com/user/ur92139946/ratings,https://www.imdb.com/user/ur92139946/reviews
142,997,https://www.imdb.com/user/ur99669382/ratings,https://www.imdb.com/user/ur99669382/reviews
143,998,https://www.imdb.com/user/ur32785137/ratings,https://www.imdb.com/user/ur32785137/reviews


In [6]:
# check if the user has a list of ratings 
movie_names_list = []
movie_ratings_list = []
user_ratings_list = []
genre_list = []
need_review_list = []
for link in users['user rating list'].values:
    r = requests.get(link)
    tree = lxml.html.fromstring(r.content)
    try:
        if len(tree.xpath('//*[@class="error_message"]')) != 0:  # when there the user doesn't have a list of ratings
            movie_names_list.append(np.nan)
            movie_ratings_list.append(np.nan)
            user_ratings_list.append(np.nan)
            genre_list.append(np.nan)
            need_review_list.append(link)
            print('the user doesnt have rating list ',link)
        else: # where the user rated a list of movies
            movie_names = tree.xpath('//h3/a[contains(@href,"/title/")]/text()') 
            movie_genres = tree.xpath('//span[@class="genre"]/text()') # if the genre include action, adventure, sci-fi, or fantasy we consider as a valid movie
            movie_ratings = tree.xpath('//div[@class="ipl-rating-star small"]/span[@class = "ipl-rating-star__rating"]/text()')
            user_ratings = tree.xpath('//div[@class="ipl-rating-star ipl-rating-star--other-user small"]/span[@class = "ipl-rating-star__rating"]/text()')
            movie_names_list.append(movie_names)
            movie_ratings_list.append(movie_ratings)
            user_ratings_list.append(user_ratings)
            genre_list.append(movie_genres)
    except: 
        print('there is an error: ',link)


the user doesnt have rating list  https://www.imdb.com/user/ur62723141/ratings
the user doesnt have rating list  https://www.imdb.com/user/ur31593185/ratings
the user doesnt have rating list  https://www.imdb.com/user/ur84265114/ratings
the user doesnt have rating list  https://www.imdb.com/user/ur101627855/ratings
the user doesnt have rating list  https://www.imdb.com/user/ur43378585/ratings
the user doesnt have rating list  https://www.imdb.com/user/ur96968871/ratings
the user doesnt have rating list  https://www.imdb.com/user/ur79943134/ratings
the user doesnt have rating list  https://www.imdb.com/user/ur100610357/ratings
the user doesnt have rating list  https://www.imdb.com/user/ur22636201/ratings
the user doesnt have rating list  https://www.imdb.com/user/ur101384076/ratings
the user doesnt have rating list  https://www.imdb.com/user/ur11320287/ratings
the user doesnt have rating list  https://www.imdb.com/user/ur92139946/ratings
the user doesnt have rating list  https://www.imd

In [7]:
need_review = [string[:-7]+"reviews" for string in need_review_list]
len(need_review)

87

In [8]:
movie_reviewed_list = []
reviewed_rating_list = []
for link in need_review:
    r = requests.get(link)
    tree = lxml.html.fromstring(r.content)
    try:
        movie_reviewed = tree.xpath('//div[@class = "lister-item-header"]/a[contains(@href,"/title/")]/text()')
        rr = tree.xpath('//span[@class="rating-other-user-rating"]/span/text()')[::2]
        reviewed_rating = list(map(int, rr))
        movie_reviewed_list.append(movie_reviewed)
        reviewed_rating_list.append(reviewed_rating)
        # no genre and overall movie rating available
    except: 
        print('there is an error: ',link)

In [9]:
len(movie_reviewed_list),movie_reviewed_list[0],reviewed_rating_list[0]

(87, ['Shazam!'], [1])

In [10]:
from itertools import compress
review_idx = pd.isnull(np.array(movie_names_list))
idx = list(compress(range(len(review_idx)), review_idx)) 

In [11]:
# build a movie corpus
# first combine the two movie list (rating, review)
for i in range(len(idx)):
    index = idx[i]
    movie_names_list[index] = movie_reviewed_list[i]
    user_ratings_list[index] = reviewed_rating_list[i]

In [18]:
user_ratings_list = [list(map(int, strings)) for strings in user_ratings_list]

In [41]:
movie_names_list[0],user_ratings_list[0] #FUCK@!

(['Shazam!'], [1])

In [73]:
len(movie_names_list)

144

In [14]:
# out pute movie_name_list, and user_ratings_list
with open('movie_reviewd_each_user.txt', 'w') as f:
    for item in movie_names_list:
        f.write("%s\n" % item)

In [21]:
with open('movie_ratings_each_user.txt', 'w') as f:
    for item in user_ratings_list:
        f.write("%s\n" % item)

In [16]:
with open('movie_ratings.txt', 'w') as f:
    for item in movie_ratings_list:
        f.write("%s\n" % item)

In [17]:
with open('movie_genres.txt', 'w') as f:
    for item in genre_list:
        f.write("%s\n" % item)

In [22]:
unique_movies, count = np.unique(list(itertools.chain.from_iterable(movie_names_list)),return_counts = True)

In [23]:
len(unique_movies) # there are 3188 unique movies reviewed in total

3188

In [233]:
# build a dataframe with movies and number of ratings
df = pd.DataFrame([unique_movies, count]).T
df.columns = ['movie_name','num_ratings']
df = df.sort_values(by = 'num_ratings', ascending = False)
df = df[df.num_ratings >= 10]
df.reset_index(inplace=True)
df

,index,movie_name,num_ratings
0,24,Game of Thrones,109
1,2078,Shazam!,65
2,10,Burn Notice,46
3,323,Avengers: Endgame,41
4,521,Captain Marvel,38
5,70,The Originals,38
6,324,Avengers: Infinity War,33
7,77,X Club Wrestling,32
8,2964,Triple Frontier,31
9,1166,How to Train Your Dragon: The Hidden World,31


In [234]:
len(df) # only 48 movies that have at least 10 ratings; there are 144 reviewers in total 

59

In [235]:
df.to_csv('df.csv')

In [207]:
# initialize the corpus
movie_corpus = np.zeros((len(movie_names_list),len(df)))
movie_corpus

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [208]:
a = list(df.movie_name.values)   # unique_movie
a[:10]

[' Game of Thrones',
 'Shazam!',
 ' Burn Notice',
 'Avengers: Endgame',
 'Captain Marvel',
 ' The Originals',
 'Avengers: Infinity War',
 ' X Club Wrestling',
 'Triple Frontier',
 'How to Train Your Dragon: The Hidden World']

In [209]:
# finally, let's build the movie corpus
# for j in range(len(movie_names_list)):
a = list(df.movie_name.values) 
for j in range(len(movie_names_list)):
# for j in range(50,70):
    b = movie_names_list[j]  # b is a list contains all movies that userj has rated
    matched_movie = np.where(np.in1d(a,b))[0]  # return index in a for value in b that match values in a
#     matched_rating = np.where(np.in1d(b,a))[0]
    mm_name = [a[mm] for mm in matched_movie]
#     print(mm_name)
    mat_rating = [b.index(f) for f in mm_name]
    matched_rating = [x for x in mat_rating if x < len(user_ratings_list[j])]
    bb = [b[hhh] for hhh in matched_rating]
    matched_movie = np.where(np.in1d(a,bb))[0]
    
#     print((matched_rating), (user_ratings_list[j]))
#     print(len(matched_rating), len(user_ratings_list[j]))
#     print(matched_movie,matched_rating)
#     print(len(matched_movie)==len(matched_rating))
    movie_corpus[j,matched_movie] = [user_ratings_list[j][mr] for mr in matched_rating]  # update the ratings for each pair of user and movie 
#     break
# print(movie_corpus)

In [210]:
movie_corpus[70:75]

array([[0., 9., 0., ..., 0., 0., 0.],
       [7., 0., 3., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [8., 0., 0., ..., 0., 0., 9.]])

In [110]:
np.savetxt("movie_corpus.txt", movie_corpus, fmt="%s") #太他妈不容易了。。。都是什么鬼。。。

In [211]:
# calculate the user similarity using the movie corpus
from sklearn.metrics.pairwise import cosine_similarity as cs
corpus_sim = cs(movie_corpus)

In [212]:
corpus_sim

array([[1.        , 0.09166985, 1.        , ..., 0.14142136, 0.        ,
        0.        ],
       [0.09166985, 1.        , 0.09166985, ..., 0.01296407, 0.        ,
        0.0140195 ],
       [1.        , 0.09166985, 1.        , ..., 0.14142136, 0.        ,
        0.        ],
       ...,
       [0.14142136, 0.01296407, 0.14142136, ..., 1.        , 0.16970563,
        0.29352601],
       [0.        , 0.        , 0.        , ..., 0.16970563, 1.        ,
        0.13108688],
       [0.        , 0.0140195 , 0.        , ..., 0.29352601, 0.13108688,
        1.        ]])

In [213]:
corpus_sim.shape

(144, 144)

In [113]:
np.savetxt("corpus_sim.txt", corpus_sim, fmt="%s") #曙光在前了。。。

In [220]:
def comparison(sim_matrix, user_index, movie_corpus = movie_corpus):
    user = sim_matrix[user_index]
    sim_user = np.where(user == max(np.delete(user, user_index, 0)))
#     print(sim_user)
    # similary user
    top_movie_index = [u.argsort()[-5:][::-1] for u in movie_corpus[sim_users]][0]
    sim_user_top_movie = [a[movie] for movie in top_movie_index]
    # self movie
    self_movie_index  = [movie_corpus[user_index].argsort()[-5:][::-1]][0]
    self_top_movie = [a[m] for m in self_movie_index]
    # overlap rate
    overlap = len(set(self_top_movie) & set(sim_user_top_movie))
#     print('self','sim_user')
    return overlap, self_top_movie, sim_user_top_movie

In [225]:
# use traditional similariry matrix
corpus_sim_performance = 0
for i in range(len(corpus_sim)):
    overlap, self_top_movie, sim_user_top_movie = comparison(corpus_sim, i)
    corpus_sim_performance+= overlap
print(corpus_sim_performance)

0
0
0
1
0
0
0
0
1
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
5
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
2
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
24


In [224]:
# use weighting similarity matrix
weight_sim_performance = 0
for i in range(144):
    overlap, self_top_movie, sim_user_top_movie =comparison(weight_sim, i)
    weight_sim_performance+= overlap
print(weight_sim_performance)

0
0
0
1
0
0
0
0
1
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
5
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
2
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
24


In [223]:
weight_sim

array([[1.        , 0.99999914, 0.99999987, ..., 0.99998963, 0.99998963,
        0.99998963],
       [0.99999914, 1.        , 0.99999964, ..., 0.99999009, 0.99999009,
        0.99999009],
       [0.99999987, 0.99999964, 1.        , ..., 0.99999014, 0.99999014,
        0.99999014],
       ...,
       [0.99998963, 0.99999009, 0.99999014, ..., 1.        , 1.        ,
        1.        ],
       [0.99998963, 0.99999009, 0.99999014, ..., 1.        , 1.        ,
        1.        ],
       [0.99998963, 0.99999009, 0.99999014, ..., 1.        , 1.        ,
        1.        ]])

# end ============================

In [138]:
# Now vadilate the result 
# 1. randomly choosing a person, find the most (2nd since the person itself will be always the most simialry)similary person
tmp = corpus_sim[3]
sim_users = np.where(tmp == max(np.delete(tmp, 3, 0)))
sim_users

(array([32]),)

In [139]:
movie_corpus[sim_users]

array([[ 0.,  6.,  0., 10., 10.,  0.,  0.,  0.,  7.,  8.,  8.,  7.,  0.,
        10.,  9.,  7.,  0.,  8., 10.,  8.,  5.,  0.,  6.,  0.,  0.,  0.,
         0.,  0.,  7., 10.,  0.,  8., 10.,  0.,  6.,  0.,  0.,  0.,  8.,
         0.,  0.,  0.,  0.,  0.,  0.,  8.,  0.,  0.,  7.,  0.,  0.,  7.,
         0.,  0.,  7.,  0.,  0.,  0.,  0.]])

In [159]:
top_movie_index = [user.argsort()[-5:][::-1] for user in movie_corpus[sim_users]][0]
# top_movie_index
sim_user_top_movie = [a[movie] for movie in top_movie_index]
sim_user_top_movie

['First Man',
 'Avengers: Endgame',
 'Spider-Man: Into the Spider-Verse',
 'A Star Is Born',
 'Captain Marvel']

In [166]:
self_movie_index  = [movie_corpus[3].argsort()[-5:][::-1]][0]
# self_movie_index
self_top_movie = [a[movie] for movie in self_movie_index]
self_top_movie

['First Man', 'Vice', 'Roma', 'Green Book', 'Captain Marvel']

In [168]:
weight_sim = np.loadtxt('weight_similarity.txt')